# Data cleaning and agregation from DB

In [ ]:
#let's start by loading the load tab from the csv file with the data loaded into the warehouse
import pandas as pd
import numpy as np

In [ ]:
load_df = pd.read_excel(io='####.xlsx',sheet_name='Load')

In [ ]:
load_df.head(10)

In [ ]:
load_df['Year'] = load_df['service_date'].dt.year
load_df['Month'] = load_df['service_date'].dt.month
load_df['woy'] = load_df['service_date'].dt.weekofyear
load_df = load_df.rename({'Unnamed: 10':'dow'},axis=1)
load_df['dow'] = load_df['service_date'].dt.dayofweek
load_df.head()

In [ ]:
#before we can start grouping we need to make sure that every day other than Sunday is assigned to the weekend of that immediate next sunday
import datetime
def set_weekend(row):
    dow = row['dow']
    date = row['service_date']
    if dow != 6:
        n = 6 - dow
        date = (date + pd.Timedelta(days=n))
    else:
        date = date
    return date
weekend_dates = load_df.apply(set_weekend,axis=1)
load_df['weekend_date'] = weekend_dates
load_df.head(10)

In [ ]:
#let's summarize this into monthly total averages for all campuses in US and then all internationals
intl_list = ['Berlin','Buenos Aires','Manila','Hong Kong']
load_us = load_df[~load_df['campus'].isin(intl_list)]
load_in = load_df[load_df['campus'].isin(intl_list)]


In [ ]:
#let's begin with us campuses
load_us_total = load_us[load_us['Year'].isin([2017,2018,2019])].groupby(['Year','Month']).agg({'attendance':np.sum,'weekend_date':'nunique'})#,('dow', lambda x: count(x) if x==1)]) 
load_us_total['Avg'] = (load_us_total['attendance']/load_us_total['weekend_date']).round(0).astype('int32')
load_us_total

In [ ]:
#let's now import the tables we summarized from the correct data
us_reference_df = pd.read_excel(io='month averages from csv.xlsx',sheet_name='US',index_col='Year')
in_reference_df = pd.read_excel(io='month averages from csv.xlsx', sheet_name = 'Intl')
us_reference_df

In [ ]:
us_reference_df.shape

In [ ]:
load_us_total.shape

In [ ]:
#let's concatenate these two 
compare_us = load_us_total.merge(us_reference_df,left_on=['Year','Month'], right_on=['Year','Month'])

In [ ]:
compare_us

In [ ]:
errors_us = compare_us[compare_us['Avg_x']!=compare_us['Avg_y']]

In [ ]:
errors_us

In [ ]:
#let's dig some more on these differences. We know that jan 17 is missing some data from dec 16 
#so we'll leave that one off, let's check the sep 17
test1 = load_us[(load_us['Month']==9)&(load_us['Year']==2017)]
test1['service_date'].value_counts()

In [ ]:
test1['weekend_date'].value_counts()

### Observation

We can see how some dates fall on a weekend that is moved to the next month. This is the case when you have an event on Saturday on the last of the month; that day is then assigned to the first weekend of the month.

**Let's then add a service month column so we can group by that column rather than the actual service date month**

In [ ]:
load_df['weekend_month'] = load_df['weekend_date'].dt.month
load_df['weekend_year'] = load_df['weekend_date'].dt.year
load_df.head()

In [ ]:
#let's now group by this column rather than the month of the actual service and see how that affects the results
#let's also filter by weekend_dateand see if that solves the issue with jan 2017
load_us = load_df[~load_df['campus'].isin(intl_list)]
load_us_total = load_us[load_us['weekend_date'].dt.year>2016].groupby(['weekend_year','weekend_month'],as_index=False).agg({'attendance':np.sum,'weekend_date':'nunique'})#,('dow', lambda x: count(x) if x==1)]) 
load_us_total['Avg'] = (load_us_total['attendance']/load_us_total['weekend_date']).round(0).astype('int32')
load_us_total

In [ ]:
#let's compare to the reference df
compare_us = load_us_total.merge(us_reference_df,left_on=['weekend_year','weekend_month'], right_on=['Year','Month'])
compare_us

In [ ]:
errors_us = compare_us[compare_us['Avg_x']!=compare_us['Avg_y']]
errors_us

# Conclusion

We have 0 errors now! that is great. This means that the data is correct after the first stage of transformation on Excel.
Let's now check the data loaded into the warehouse to verify is working fine.


In [ ]:
#let's also do international campuses and add the two of them
load_in = load_df[load_df['campus'].isin(intl_list)]
load_in_total = load_in[load_in['weekend_date'].dt.year>2016].groupby(['weekend_year','weekend_month'],as_index=False).agg({'attendance':np.sum,'weekend_date':'nunique'})#,('dow', lambda x: count(x) if x==1)]) 
load_in_total['Avg'] = (load_in_total['attendance']/load_in_total['weekend_date']).round(0).astype('int32')
load_in_total

In [ ]:
compare_in = load_in_total.merge(in_reference_df,left_on=['weekend_year','weekend_month'], right_on=['Year','Month'])
compare_in

In [ ]:
errors_in = compare_in[compare_in['Avg_x']!=compare_in['Avg_y']]
errors_in

##  Include Volunteers

It seems that the discrepancies between the load and the Intl tabs are due to the the volunteers. On the Intl tab, the volunteers were included in the count. They were not incldued on the Load tab. Let's bring in the volunteers count, add them to the international and see the differences

In [ ]:
in_volunteers = pd.read_excel('month averages from csv.xlsx',sheet_name='Volunteers')
in_volunteers.head()

In [ ]:
total_in = compare_in.merge(in_volunteers,left_on=['Year','Month'],right_on=['Year','Month'])
total_in['TotalLoadWithVolunteers'] = total_in['Avg_x']+total_in['Avg']
total_in

In [ ]:
errors_in = total_in[total_in['Avg_y']!=total_in['TotalLoadWithVolunteers']]
errors_in

## Conclusion

We can see that the numbers are different for less than 0.001%. That is probably due to the rounding up on the calculations and is a very acceptable margin of error. Let's now do a Load Dataframe with all the DF results so we can later compare to the calculations from the Warehouse

In [ ]:
load_totals = load_us_total.merge(total_in[['weekend_year','weekend_month','Avg_x','TotalLoadWithVolunteers']],left_on=['weekend_year','weekend_month'],right_on=['weekend_year','weekend_month']).rename({'TotalLoadWithVolunteers':'IntlWithVolunteers'},axis=1)
load_totals

In [ ]:
load_totals['All Campuses with Volunteers']=load_totals['Avg']+load_totals['IntlWithVolunteers']
load_totals['All Campuses']=load_totals['Avg']+load_totals['Avg_x']

load_totals

In [ ]:
total_reference = us_reference_df.merge(in_reference_df,left_on=['Year','Month'],right_on=['Year','Month'])

In [ ]:
total_reference['Totals'] = total_reference['Avg_x']+total_reference['Avg_y']
total_reference

In [ ]:
compare_all = total_reference.merge(load_totals[['weekend_year','weekend_month','All Campuses with Volunteers']],left_on=['Year','Month'],right_on=['weekend_year','weekend_month'])
errors_all_campuses = compare_all[compare_all['Totals']!=compare_all['All Campuses with Volunteers']]
errors_all_campuses['diff']=errors_all_campuses['Totals']-errors_all_campuses['All Campuses with Volunteers']
errors_all_campuses

In [ ]:
writer = pd.ExcelWriter('Totals from Load Tab.xlsx',engine='xlsxwriter')
load_totals.to_excel(writer,sheet_name = 'LoadTotals')
writer.save()